In [ ]:
import numpy as np
x = np.arange(1, 4)
y = np.array([2,4,6])
np.sum(x*x)*0.5*np.sum(y*y)*0.5

196.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import numpy as np
import cv2
import os
import csv
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_dataset():
    # Load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    # Reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))

    trainX = np.where(trainX > 0, 255, 0)
    testX = np.where(testX > 0, 255, 0)
    # One-hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

def prep_pixels(train, test):
    # Convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # Normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    return train_norm, test_norm

def define_model():
    model = Sequential()
    model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(28, 28, 1), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (3, 3), activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))

    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def run():
    # Load dataset
    trainX, trainY, testX, testY = load_dataset()
    # Prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # Define model
    model = define_model()

    # Create data generator
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=False,
        brightness_range=[0., 1.5]
    )

    # Fit model
    model.fit(datagen.flow(trainX, trainY, batch_size=128), epochs=10, validation_data=datagen.flow(testX, testY), verbose=1)
    # Save model
    model.save('final_model.h5')

In [ ]:
run()

In [ ]:
def create_dataset_from_folder(folder):
    images = []
    filenames = []
    def load_image(filename):
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if img is None:
          return None
        img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_AREA)
        img = img.reshape(28, 28, 1)
        copy = img.copy()
        mean = cv2.mean(copy)[0]
        if mean < 100:
          img[copy > mean] = 255
          img[copy <= mean] = 0
        else:
          img[copy > mean] = 0
          img[copy <= mean] = 255
        img = img.astype('float32') / 255.0
        return img

    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        img = load_image(filepath)
        if img is not None:
            images.append(img)
            filenames.append(filename)

    return filenames, np.array(images)

def save_predictions_to_csv(filenames, predictions, csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for filename, prediction in zip(filenames, predictions):
            writer.writerow([filename, prediction])

model = load_model('final_model.h5')

root_folder = './data'
filenames, test_dataset = create_dataset_from_folder(root_folder)

predictions = model.predict(test_dataset)
predicted_labels = np.argmax(predictions, axis=1)

csv_filename = 'submission.csv'
save_predictions_to_csv(filenames, predicted_labels, csv_filename)

In [ ]:
plt.figure(figsize=(28, 28))
start = 1000
end = 1090
for i in range(start, end):
    plt.subplot(30, 3, i - start + 1)
    plt.imshow(test_dataset[i], cmap='gray')
    plt.title(f"predict: {predicted_labels[i]}")
    plt.axis('off')
plt.show()